# 0. Set-up
Mount drive, load packages, load data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

In [0]:
from keras import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import random
from keras.layers import ZeroPadding2D, Convolution2D,Activation
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.utils import np_utils, plot_model
import pickle
from random import shuffle
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout, Input
from keras import *
from keras.optimizers import *
from keras import metrics
from keras.callbacks import *
from sklearn.metrics import accuracy_score, confusion_matrix
import h5py # to save best models

In [0]:
# Get own metric (FPR)
import keras.backend as K
def FPR(y_true,y_pred):
  """
  param:
  y_pred - Predicted labels
  y_true - True labels 
  Returns:
  False Positive Rate
  """
  neg_y_true = 1 - y_true
  neg_y_pred = 1 - y_pred
  fp = K.sum(neg_y_true * y_pred)
  tn = K.sum(neg_y_true * neg_y_pred)
  fpr = fp / (tn + fp + K.epsilon())
  
  return(fpr)

In [0]:
def make_model_own(input_shape):
  '''Builds a shallow CNN with input_shape = input_shape'''
  own_model = Sequential()
  own_model.add(Conv2D(32, (3, 3), activation='relu',
                   input_shape=input_shape))
  own_model.add(MaxPooling2D(pool_size=(2, 2)))
  own_model.add(Conv2D(32, (3,3), activation = 'relu'))
  own_model.add(MaxPooling2D(pool_size = (2,2)))
  own_model.add(Conv2D(64, (3,3), activation='relu'))
  own_model.add(MaxPooling2D(pool_size=(2, 2)))
  own_model.add(Conv2D(64, (3,3), activation = 'relu'))
  own_model.add(MaxPooling2D(pool_size=(2, 2)))
  own_model.add(Flatten())
  own_model.add(Dense(128, activation='relu'))
  own_model.add(Dense(128, activation = 'relu'))
  own_model.add(Dropout(0.5))
  own_model.add(Dense(2, activation='sigmoid')) 
  return(own_model)



def compile_and_run(Model, X, y, val_X, val_y, save_path, batch_size = 32, loss = 'binary_crossentropy', 
                   optimizer = optimizers.Adam(lr = 0.001, amsgrad = True), epochs = 50):
  '''Runs the shallow CNN. Parameters:
  Model: an uncompiled model to run
  X: the train data
  y: The train labels
  val_X: the validation data
  val_y: the validation labels
  save_path: a path to where the best model weights are saved
  batch_size: the batch size for training/validation (default: 32)
  loss: the loss function to use (default: binary_crossentropy)
  optimizer: the optimizer to use (default: Adam(lr = 0.001, amsgrad = True))
  epochs: the number of epochs to run for (default: 50)'''
  
  # Define a data generator
  train_gen = ImageDataGenerator(rotation_range = 30,
                            width_shift_range = 0.1,
                            height_shift_range = 0.1,
                            horizontal_flip = True,
                            shear_range = 0.02,
                            )
  
  train_gen.fit(X)
  train_generator = train_gen.flow(X,y,batch_size = batch_size, shuffle = True)
  
  # Compile the model
  Model.compile(loss= loss,
              optimizer = optimizer,
              metrics=['accuracy', FPR])
  
  # Create callbacks; early stopping and saving model weights
  cb = Callback()
  es = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
  mc = ModelCheckpoint(save_path,monitor='val_loss', 
                       mode='min', save_best_only=True, save_weights_only = True, verbose = 1)
  
  
  # Train the model
  history = Model.fit_generator(train_generator, steps_per_epoch = len(X)/batch_size, epochs = epochs, 
                      callbacks = [es,mc], validation_data = (val_X, val_y))
  
  return(history, Model)

In [0]:
# Load data
X = pickle.load(open('gdrive/My Drive/Data/preprocessed/X.txt','rb'))
y = pickle.load(open('gdrive/My Drive/Data/preprocessed/y.txt','rb'))
Holdout_X = pickle.load(open('gdrive/My Drive/Data/preprocessed/valX.txt','rb'))
Holdout_y = pickle.load(open('gdrive/My Drive/Data/preprocessed/valy.txt','rb'))

In [0]:
# split the holdout data into a validation and test set
val_x = np.concatenate((Holdout_X[:100],Holdout_X[500:600],Holdout_X[1000:1100]))
val_y = np.concatenate((Holdout_y[:100],Holdout_y[500:600],Holdout_y[1000:1100]))
Holdout_X = np.concatenate((Holdout_X[100:500],Holdout_X[600:1000],Holdout_X[1100:1500]))
Holdout_y = np.concatenate((Holdout_y[100:500],Holdout_y[600:1000],Holdout_y[1100:1500]))
                           

# Training models

## Shallow self-built CNN

In [0]:
own_model = make_model_own((224,224,3))
own_model.summary()

In [0]:
plot_model(own_model,'gdrive/My Drive/Deep Learning/shallow.png')

In [0]:
random.seed(0)
savepath = 'gdrive/My Drive/Data/Output/model_weights/final_own_basic_v2.hdf5'
history, model = compile_and_run(own_model,X,y,val_x,val_y,savepath,epochs = 50)

In [0]:
own_evals = own_model.evaluate(Holdout_X, Holdout_y)
own_evals

## VGG16 Model
Is done slightly differently as it is not a Sequential() type.

In [0]:
vgg16_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (224,224,3))
for layer in vgg16_base.layers:
    layer.trainable = False
    
# Add output layers
x = Flatten()(vgg16_base.output)
x = Dense(2, activation = 'sigmoid')(x)


vgg16_model = Model(inputs = vgg16_base.input,outputs = x)
vgg16_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg16_model.summary()

In [0]:
plot_model(vgg16_model,'gdrive/My Drive/Deep Learning/vgg16.png')

In [0]:
cb_vgg16 = Callback()
es_vgg16 = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5,
                        restore_best_weights = True)
mc_vgg16 = ModelCheckpoint('gdrive/My Drive/Data/best_model_vgg16.hdf5',
                           monitor='val_loss', mode='min', save_best_only=True, save_weights_only = True, verbose = 1)


In [0]:
datagen = ImageDataGenerator(rotation_range = 30,
                             width_shift_range = 0.1,
                             height_shift_range = 0.1,
                             horizontal_flip = True,
                             shear_range = 0.01,
                            )

datagen.fit(X)

In [0]:
random.seed(0)
history_vgg16_basic = vgg16_model.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vgg16,mc_vgg16], validation_data = [val_x,val_y])

In [0]:
vgg16_evals = vgg16_model.evaluate(Holdout_X, Holdout_y)
vgg16_evals

## VGG-Face

In [0]:
def VGG_face():
  '''
  Make the vgg face model and load weights (these need to be downloaded)'''
  face_model = Sequential()
  face_model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
  face_model.add(Convolution2D(64, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(64, (3, 3), activation='relu'))
  face_model.add(MaxPooling2D((2,2), strides=(2,2)))

  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(128, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(128, (3, 3), activation='relu'))
  face_model.add(MaxPooling2D((2,2), strides=(2,2)))

  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(256, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(256, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(256, (3, 3), activation='relu'))
  face_model.add(MaxPooling2D((2,2), strides=(2,2)))

  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(MaxPooling2D((2,2), strides=(2,2)))

  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(ZeroPadding2D((1,1)))
  face_model.add(Convolution2D(512, (3, 3), activation='relu'))
  face_model.add(MaxPooling2D((2,2), strides=(2,2)))

  face_model.add(Convolution2D(4096, (7, 7), activation='relu'))
  face_model.add(Dropout(0.5))
  face_model.add(Convolution2D(4096, (1, 1), activation='relu'))
  face_model.add(Dropout(0.5))
  face_model.add(Convolution2D(2622, (1, 1)))
  face_model.add(Flatten())
  face_model.load_weights('gdrive/My Drive/Data/vgg_face_weights.h5')
  return(face_model)

In [0]:
# Ensure the convolutional layers are frozen

vgg_face_model = VGG_face()

for layer in vgg_face_model.layers:
  layer.trainable = False

vgg_face_model.add(Dense(2, activation = 'sigmoid'))

vgg_face_model.summary()

In [0]:
plot_model(vgg_face_model,'gdrive/My Drive/Deep Learning/vggface.png')

In [0]:
vgg_face_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

In [0]:
es_vggface = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vggface = ModelCheckpoint('gdrive/My Drive/Data/best_model_vggface.h5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)


In [0]:
random.seed(0)
history_vgg_face_basic = vgg_face_model.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vggface,mc_vggface], validation_data = [val_x,val_y])

In [0]:
vgg_face_evals = vgg_face_model.evaluate(Holdout_X, Holdout_y)
vgg_face_evals

## VGG16 Unfreezing layers

In [0]:
vgg16_ft_base = VGG16(weights = 'imagenet', include_top = False, input_shape = (224,224,3))
for layer in vgg16_ft_base.layers[:-2]:
    layer.trainable = False
    
# Add output layers
x = Flatten()(vgg16_ft_base.output)
x = Dense(2, activation = 'sigmoid')(x)


vgg16_ft_model = Model(inputs = vgg16_ft_base.input,outputs = x)
vgg16_ft_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg16_ft_model.summary()

In [0]:
es_vgg16_ft = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vgg16_ft = ModelCheckpoint('gdrive/My Drive/Data/best_model_vgg16_ft.h5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)

In [0]:
random.seed(0)


history_vgg16_ft = vgg16_ft_model.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vgg16_ft,mc_vgg16_ft], validation_data = [val_x,val_y])

In [0]:
vgg16_ft_model.load_weights('gdrive/My Drive/Data/best_model_vggface_ft.h5')
vgg16_ft_evals = vgg16_ft_model.evaluate(Holdout_X, Holdout_y)
vgg16_ft_evals

## VGG-Face Finetuning

In [0]:
vgg_face_model = VGG_face()

for layer in vgg_face_model.layers[:-2]:
  layer.trainable = False

vgg_face_model.add(Dense(2, activation = 'sigmoid'))

vgg_face_model.summary()

In [0]:
vgg_face_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

es_vggface = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vggface = ModelCheckpoint('gdrive/My Drive/Data/best_model_vggface_ft.hdf5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)
random.seed(0)
history_vgg_face_ft = vgg_face_model.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vggface,mc_vggface], validation_data = [val_x,val_y])

# Testing and Evaluating all models

I test on: the holdout data, a seperate unseen dataset containing images from different folders (so completely new persons) and a dataset containing only images of people between 15-32 years of age. In the next section, I also try training/testing the best model on the 15-32 years dataset. 

**Note that accuracies reported by Keras are often incorrect! These were calculated manually for the report.**

## Holdout data

In [0]:
# Load weights for each model and compile them
# Own model
own_model = make_model_own((224,224,3))
own_model.load_weights('gdrive/My Drive/Data/Output/model_weights/final_own_basic_v2.hdf5')
own_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

# VGG16 Model
vgg16_base = VGG16(include_top = False, input_shape = (224,224,3))
    
 # Add output layers
x = Flatten()(vgg16_base.output)
x = Dense(2, activation = 'sigmoid')(x)


vgg16_model = Model(inputs = vgg16_base.input,outputs = x)
vgg16_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg16_model.load_weights('gdrive/My Drive/Data/best_model_vgg16.hdf5')


# VGG-Face Model
vgg_face_model = VGG_face()
vgg_face_model.add(Dense(2, activation = 'sigmoid'))
vgg_face_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg_face_model.load_weights('gdrive/My Drive/Data/best_model_vggface.h5')

# VGG16 Finetuned
vgg16_ft_base = VGG16(include_top = False, input_shape = (224,224,3))
    
 # Add output layers
x = Flatten()(vgg16_ft_base.output)
x = Dense(2, activation = 'sigmoid')(x)


vgg16_ft_model = Model(inputs = vgg16_ft_base.input,outputs = x)
vgg16_ft_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg16_ft_model.load_weights('gdrive/My Drive/Data/best_model_vgg16_ft.h5')


# VGG-Face Finetuned
vgg_face_model_ft = VGG_face()
vgg_face_model_ft.add(Dense(2, activation = 'sigmoid'))
vgg_face_model_ft.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

vgg_face_model_ft.load_weights('gdrive/My Drive/Data/best_model_vggface_ft.hdf5')


In [0]:
# Evaluate each model
unseen_evals = []
unseen_evals.append(own_model.evaluate(Holdout_X, Holdout_y))
unseen_evals.append(vgg16_model.evaluate(Holdout_X, Holdout_y))
unseen_evals.append(vgg16_ft_model.evaluate(Holdout_X, Holdout_y))
unseen_evals.append(vgg_face_model.evaluate(Holdout_X, Holdout_y))
unseen_evals.append(vgg_face_model_ft.evaluate(Holdout_X, Holdout_y))
unseen_evals

In [0]:
own_preds = vgg_face_model_ft.predict_classes(Holdout_X)
#own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,Holdout_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

In [0]:
own_preds = vgg_face_model.predict_classes(Holdout_X)
#own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,Holdout_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

## Unseen data

In [0]:
Unseen_X = pickle.load(open('gdrive/My Drive/Data/preprocessed/x_D.txt','rb'))
Unseen_y = pickle.load(open('gdrive/My Drive/Data/preprocessed/y_D.txt','rb'))

In [0]:
own_preds = vgg_face_model_ft.predict_classes(Unseen_X)
#own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,Unseen_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

In [0]:
own_preds = vgg16_ft_model.predict(Unseen_X)
own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,Unseen_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

In [0]:
unseen_evals = []
unseen_evals.append(own_model.evaluate(Unseen_X, Unseen_y))
unseen_evals.append(vgg16_model.evaluate(Unseen_X, Unseen_y))
unseen_evals.append(vgg16_ft_model.evaluate(Unseen_X, Unseen_y))
unseen_evals.append(vgg_face_model.evaluate(Unseen_X, Unseen_y))
unseen_evals.append(vgg_face_model_ft.evaluate(Unseen_X, Unseen_y))
unseen_evals

## Data of those between 15-32

In [0]:
_15_X = pickle.load(open('gdrive/My Drive/Data/preprocessed/_15_32_D_X.txt','rb'))
_15_y = pickle.load(open('gdrive/My Drive/Data/preprocessed/_15_32_D_y.txt','rb'))

In [0]:
_15_y = _15_y[:round(len(_15_y)*.99)]

In [0]:
vgg16_ft_model.evaluate(_15_X,_15_y)

In [0]:
own_preds = vgg_face_model.predict_classes(_15_X)
#own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,_15_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

In [0]:
own_preds = vgg_face_model_ft.predict_classes(_15_X)
#own_preds = np.argmax(own_preds,axis = 1)
con_own = confusion_matrix(own_preds,_15_y[:,1])
tn, fp, fn, tp = con_own.ravel()
tn, fp, fn, tp

In [0]:
_15_evals = []
_15_evals.append(own_model.evaluate(_15_X, _15_y))
_15_evals.append(vgg16_model.evaluate(_15_X, _15_y))
_15_evals.append(vgg16_ft_model.evaluate(_15_X, _15_y))
_15_evals.append(vgg_face_model.evaluate(_15_X, _15_y))
_15_evals.append(vgg_face_model_ft.evaluate(_15_X, _15_y))
_15_evals

In [0]:
vgg_face_model.predict(_15_X)

## Wrong predictions

In [0]:
predictions = vgg_face_model.predict_classes(_15_X)

In [0]:
x = 0
for i in range(len(predictions)):
  if predictions[x] != int(_15_y[x,1]):
    print(x)
  x += 1

In [0]:
_15_y[700],predictions[727]

In [0]:
plt.imshow(_15_X[727])

# VGG-Face Layer Output

In [0]:
vgg_face_model = VGG_face()
vgg_face_model.add(Dense(2, activation = 'sigmoid'))
vgg_face_model.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])
vgg_face_model.load_weights('gdrive/My Drive/Data/best_model_vggface.h5')

In [0]:
vgg_face_outputs = [layer.output for layer in vgg_face_model.layers[:36]]

activation_model = models.Model(inputs = vgg_face_model.input,outputs = vgg_face_outputs)

In [0]:
activations = activation_model.predict(_18_X[250:251])

In [0]:
first_layer_activation = activations[0]
first_layer_activation.shape

In [0]:
third_layer_activation = activations[25]

In [0]:
layer_names = []
for layer in vgg_face_model.layers[30:36]:
  layer_names.append(layer.name)
  
images_per_row = 2

for layer_name, layer_activation in zip(layer_names,activations):
  n_features = layer_activation.shape[-1] # number of features in feature map
  size = layer_activation.shape[1]
  n_cols = n_features // images_per_row # tiles the activation channels in this matrix
  display_grid = np.zeros((size * n_cols, images_per_row * size))
  for col in range(n_cols):
    for row in range(images_per_row):
      channel_image = layer_activation[0,
                                      :, :,
                                      col * images_per_row + row]
      channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
      channel_image /= channel_image.std()
      channel_image *= 64
      channel_image += 128
      channel_image = np.clip(channel_image, 0, 255).astype('uint8')
      display_grid[col * size : (col + 1) * size, # Displays the grid
                   row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [0]:
vgg_face_model.summary()

# VGG-Face training and testing on 18-32 age group


In [0]:
_X = pickle.load(open('gdrive/My Drive/Data/preprocessed/_18_32_X_1of2.txt','rb'))
_y = pickle.load(open('gdrive/My Drive/Data/preprocessed/_18_32_y_1of2.txt','rb'))

In [0]:
_y = _y[:round(len(_y)*.99)]
_X.shape,_y.shape

In [0]:
# Get 250 random validation samples:
random_samples = []
for i in range(250):
  random_samples.append(random.randint(0,3207))

In [0]:
val_y = _y[random_samples]
val_X = _X[random_samples]

In [0]:
val_X.shape

In [0]:
train_X = np.delete(_X,random_samples, axis = 0)
train_y = np.delete(_y,random_samples, axis = 0)
train_X.shape

In [0]:
datagen = ImageDataGenerator(rotation_range = 30,
                             width_shift_range = 0.1,
                             height_shift_range = 0.1,
                             horizontal_flip = True,
                             shear_range = 0.01,
                            )

datagen.fit(train_X)

In [0]:
vgg_face_

In [0]:
vgg_face_basic_18 = VGG_face()
for layer in vgg_face_basic_18.layers:
  layer.trainable = False

vgg_face_basic_18.add(Dense(2, activation = 'sigmoid'))
vgg_face_basic_18.compile(loss= 'binary_crossentropy',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

es_vggface = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vggface = ModelCheckpoint('gdrive/My Drive/Data/best_model_vggface_18.hdf5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)

random.seed(0)
vgg_face_basic_18_hist = vgg_face_basic_18.fit_generator(datagen.flow(train_X,train_y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(train_X) / 32,epochs = 50,
                       callbacks = [es_vggface,mc_vggface], validation_data = [val_X,val_y])

In [0]:
vgg_face_basic_18.evaluate(_18_X,_18_y)

In [0]:
sum(_y)

# With Hinge loss to reduce FPR

In [0]:
vgg_face_hinge_ft = VGG_face()
for layer in vgg_face_hinge_ft.layers[:-2]:
  layer.trainable = False

vgg_face_hinge_ft.add(Dense(2, activation = 'sigmoid'))

vgg_face_hinge_ft.compile(loss= 'categorical_hinge',
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

es_vggface = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vggface = ModelCheckpoint('gdrive/My Drive/Data/best_model_vggface_ft_hinge.hdf5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)
random.seed(0)
history_vgg_face_ft_hinge = vgg_face_hinge_ft.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vggface,mc_vggface], validation_data = [val_X,val_y])

In [0]:
evals = vgg_face_hinge_ft.evaluate(Holdout_X, Holdout_y)
evals

# Custom loss function which penalizes false positives

Does not work yet

In [0]:
import keras.backend as K
def binary_recall_specificity(y_true, y_pred, recall_weight, spec_weight):

    TN = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 0)
    TP = np.logical_and(K.eval(y_true) == 1, K.eval(y_pred) == 1)

    FP = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 1)
    FN = np.logical_and(K.eval(y_true) == 1, K.eval(y_pred) == 0)

    # Converted as Keras Tensors
    TN = K.sum(K.variable(TN))
    FP = K.sum(K.variable(FP))

    specificity = TN / (TN + FP + K.epsilon())
    recall = TP / (TP + FN + K.epsilon())

    return(1.0 - (recall_weight*recall + spec_weight*specificity))
  
# Our custom loss' wrapper
def custom_loss(recall_weight, spec_weight):

    def recall_spec_loss(y_true, y_pred):
        return(binary_recall_specificity(y_true, y_pred, recall_weight, spec_weight))

    # Returns the (y_true, y_pred) loss function
    return(recall_spec_loss)

In [0]:
vgg_face_ft_cl = VGG_face()
for layer in vgg_face_ft_cl.layers[:-2]:
  layer.trainable = False

vgg_face_ft_cl.add(Dense(2, activation = 'sigmoid'))


loss = custom_loss(0.25,0.75)

vgg_face_ft_cl.compile(loss= loss,
              optimizer = optimizers.Adam(lr = 0.001, amsgrad = True),
              metrics=['accuracy', FPR])

es_vggface = EarlyStopping(monitor = 'val_loss',mode = 'min',verbose = 1, patience = 5, restore_best_weights = True)
mc_vggface = ModelCheckpoint('gdrive/My Drive/Data/best_model_vggface_ft_hinge.hdf5',monitor='val_loss', mode='min',
                             save_best_only=True, save_weights_only = True, verbose = 1)

loss = custom_loss()

random.seed(0)
hist_vgg_face_ft_cl = vgg_face_hinge_ft.fit_generator(datagen.flow(X,y,batch_size=32,shuffle = True),
                       steps_per_epoch = len(X) / 32,epochs = 50,
                       callbacks = [es_vggface,mc_vggface], validation_data = [val_x,val_y])

# save_path_ft_cl = 'gdrive/My Drive/Data/best_model_vggface_ft_cl.hdf5'
# history, vgg_face_ft_cl = compile_and_run(vgg_face_ft_cl, X, y, val_x, val_y, save_path_ft_cl, batch_size = 32, loss = custom_loss(0.25,0.75), 
#                    optimizer = optimizers.Adam(lr = 0.001, amsgrad = True), epochs = 50)

